# Introduction
The purpose of this notebook is to perform some basic analytics on the Yelp Academic Dataset. 

Data ingestion is handled using "query_create.py", "load_database.py" and "create_views.py". A PostgreSQL database was chosen since it has good support for JSON data, and SQL seems natural for the types of queries we will be performing.

In [110]:
# Import libraries
import os
import sys
import logging

# Import psycopg2 for connection to postgres via Python
try:
    import psycopg2 as pg
    import psycopg2.extras
except:
    print( "Install psycopg2")
    exit(123)

# Database connection   
PG_CONN_STRING = "dbname='postgres' port='5432' user='postgres' password='phludphlud'"
dbconn = pg.connect(PG_CONN_STRING)
cursor = dbconn.cursor()

# Question 2
Top 10 restaurants in Toronto with the highest popularity. Popularity is defined by number of reviews.

In [2]:
cursor.execute("""
               SELECT count(r.review_id), b.name FROM 
               (SELECT * FROM business_view 
               WHERE business_view.city = 'Toronto'
               ) b
               INNER JOIN review_view r
               ON b.business_id = r.business_id
               GROUP BY b.name
               order by count DESC
               LIMIT 10;
               """)
print(cursor.fetchall())

[(1837, 'Pai Northern Thai Kitchen'), (1513, 'Banh Mi Boys'), (1443, 'Starbucks'), (1336, 'Khao San Road'), (1306, 'KINKA IZAKAYA ORIGINAL'), (1061, 'Terroni'), (1056, "The Burger's Priest"), (1048, 'Seven Lives Tacos Y Mariscos'), (967, 'La Carnita'), (940, "Jack Astor's Bar & Grill")]


# Question 3
Number of Canadian residents who reviewed Mon Ami Gabi in past year. Since user's location of residence is not provided, the location where they have the most reviews will be assumed to be their place of residence.

In [113]:
# Reviews grouped by user and then by province
cursor.execute("""
                WITH q as (
                WITH a as (
                SELECT b.state as state, r.user_id as user_id, COUNT(*) as count FROM review_view r
                LEFT JOIN business_view b
                ON r.business_id = b.business_id                
                GROUP BY r.user_id, b.state)                
                SELECT * FROM 
                (SELECT ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY count) AS r,
                a.*
                FROM a) x
                WHERE x.r <= 1)
                SELECT count(*) FROM review_view r
                LEFT JOIN q
                ON r.user_id = q.user_id
                WHERE r.business_id = '4JNXUYY8wbaaDmk3BPzlWw' AND r.date > '2017-08-21' AND q.state IN ('BC', 'AB', 'ON', 'QC')
                """)
cursor.fetchall()

[(19,)]

# Question 4
Most common words in reviews of Chipotle Mexican Grill.

In [117]:
cursor.execute("""
               SELECT r.text FROM 
               business_view b
               INNER JOIN review_view r
               ON b.business_id = r.business_id
               WHERE b.name = 'Chipotle Mexican Grill'
               
               
               """)
reviews = cursor.fetchall()

word_count = {}

for i in reviews:
    text = i[0]
    text = text.translate({ord(c): None for c in '!@#$.,"'})
    text = text.split()
    for word in text:
        if word in word_count:
            word_count[word] += 1
        else:
            word_count[word] = 1

print(sorted(word_count, key=word_count.__getitem__)[-10:])

['in', 'it', 'was', 'of', 'is', 'a', 'to', 'and', 'I', 'the']


# Question 5
Percentage of viewers who reviewed Mon Ami Gabi who also reviewed at least 10 restaurants in Toronto.

In [8]:
cursor.execute("""
               WITH mag_reviewers as
               (
               SELECT r.user_id as u_id FROM review_view r
               LEFT JOIN users_view u
               ON r.user_id = u.user_id
               WHERE r.business_id = '4JNXUYY8wbaaDmk3BPzlWw'
               )              
               
               SELECT COUNT (DISTINCT u_id) FROM mag_reviewers;                            
               """)

number_mag_reviewers = cursor.fetchone()[0]
print("The number of total Mon Ami Gabi reviewers is {}.".format(number_mag_reviewers))
cursor.execute("""
               WITH mag_reviewers as
               (
               SELECT r.user_id as u_id, u.name as name FROM review_view r
               LEFT JOIN users_view u
               ON r.user_id = u.user_id
               WHERE r.business_id = '4JNXUYY8wbaaDmk3BPzlWw'
               ) 
               SELECT count(*) FROM mag_reviewers
               INNER JOIN
               (
               SELECT count(u.user_id), u.user_id AS u_id
               FROM users_view u
               LEFT JOIN 
               (SELECT r.user_id AS u_id FROM review_view r
               LEFT JOIN business_view b
               ON r.business_id = b.business_id  
               WHERE b.state = 'ON'
               ) e1
               ON u.user_id = e1.u_id
               GROUP BY u.user_id
               HAVING count(u.name) > 10
               ) ont_reviews
               ON mag_reviewers.u_id = ont_reviews.u_id
               
               """)

number_mag_reviewers_ON10 = cursor.fetchone()[0]
print("The number of Mon Ami Gabi reviewers who also reviewed at least 10 restaurants in Ontario is {}.".format(number_mag_reviewers_ON10))

print("The percentage is {:.2f}%".format(number_mag_reviewers_ON10/number_mag_reviewers * 100))

The number of total Mon Ami Gabi reviewers is 7968.
The number of Mon Ami Gabi reviewers who also reviewed at least 10 restaurants in Ontario is 70.
The percentage is 0.88%


# Question 6
Two additional analytics that could be helpful to Business Owners:

1. Does the previous number of check-ins at a given time predict the future number of check-ins or customers at the same time? (Helps the business with inventory, staffing, etc.)

2. What is the average review of a certain type of food (i.e. Mexican, Chinese, etc.) given neighbourhood or latitude/longitude? (Allows a business to identify possible locations for opening another store)
